In [2]:
import torch

In [7]:
tensor_0d = torch.tensor(0)
tensor_1d = torch.tensor([1,2,3])
tensor_2d = torch.tensor([[1,2,3],
                          [4,5,6]])
tensor_3d = torch.tensor([[[1,2,9],
                           [3,4,9]],
                          [[5,6,0],
                           [7, 8,0]]])
print(tensor_0d.shape, tensor_1d.shape, tensor_2d.shape, tensor_3d.shape)

torch.Size([]) torch.Size([3]) torch.Size([2, 3]) torch.Size([2, 2, 3])


In [8]:
tensor_3d.dtype

torch.int64

In [9]:
float_tensor = torch.tensor([0.4, 0.8])
float_tensor.dtype

torch.float32

In [10]:
big_float = float_tensor.to(torch.float64)
big_float.dtype

torch.float64

In [13]:
tensor_2d.reshape(3,2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [14]:
tensor_2d.view(3,2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [12]:
tensor_2d.T

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [15]:
tensor_2d @ tensor_2d.T

tensor([[14, 32],
        [32, 77]])

## AutoGrad

In [18]:
import torch.nn.functional as F
from torch.autograd import grad

In [19]:
y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0],requires_grad=True)
z = x1 * w1 + b

a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

grad_w1 = grad(loss, w1, retain_graph=True)
grad_b = grad(loss, b, retain_graph=True)
print(grad_w1)
print(grad_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [20]:
loss.backward()

In [21]:
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


## MLP

In [23]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_output):
        super().__init__()
        self.layers = torch.nn.Sequential(
            #1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            #2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            #output layer
            torch.nn.Linear(20, num_output),
        )
    
    def forward(self, x):
        logits = self.layers(x)
        return logits
          

In [24]:
model = NeuralNetwork(50, 3)

In [ ]:
model

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)

In [26]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [27]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [28]:
X = torch.rand((1,50))
out = model(X)
out

tensor([[ 0.0555,  0.1039, -0.1024]], grad_fn=<AddmmBackward0>)

In [29]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3444, 0.3615, 0.2941]])


## Data Loader